In [21]:
# Librerias necesarias
import numpy as np
import pandas as pd
import matplotlib.pylab as pt
from sklearn import metrics
import seaborn as sns
import random
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn import svm
%matplotlib inline

In [22]:
# Toma un conjunto de datos y lo separa en conjunto de datos positivos y conjunto de datos negtivos
def split_positive_negative(dataframe):
    positive_df = dataframe[dataframe['Dx:Cancer'] == 1]
    negative_df = dataframe[dataframe['Dx:Cancer'] == 0]
    return positive_df, negative_df

# Toma un conjunto de datos y lo separa en x (variable cancer) e y (resto de variables), y lo convierte a np.array
def split_xy(dataframe):
    df_x = dataframe.copy()
    df_x.pop("Dx:Cancer")
    y = dataframe["Dx:Cancer"].to_numpy()
    x = df_x.to_numpy()
    return x, y

## # Random Forest Classifier

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html


### CROSS VALIDATION

In [23]:
from sklearn.model_selection import cross_val_score

num_folds = 10

# Cogemos el conjunto de datos bueno (clases balanceadas)
df = pd.read_excel("datos.xlsx")
positive_df, negative_df = split_positive_negative(df)
for i in range(int(negative_df.shape[0]/positive_df.shape[0])):
    df = pd.concat([df, positive_df])
x, y = split_xy(df)
# Creamos el modelo de random forest
model = RandomForestClassifier(n_estimators=100, max_features='auto')
# Lo entrenamos y evaluamos mediante cross validation
scores = cross_val_score(model, x, y, cv=num_folds, scoring='f1')
scores = np.mean(scores)

### CROSS VALIDATION PARA ELEGIR HIPERPARAMETROS DEL MODELO

Vamos a probar las siguientes combinaciones de hiperparametros:
- Numero de arboles: 100, 500 o 1000
- Criterio: gini o entropia
- Numero maximo de variables empleadas en cada arbol de decision: auto, raiz(numero de variables) o log2(numero de variables

In [24]:
num_trees = [100, 500, 1000]
criterion = ['gini', 'entropy']
max_features = ['auto', 'sqrt', 'log2']

num_folds = 10

In [25]:
df = pd.read_excel("datos.xlsx")
positive_df, negative_df = split_positive_negative(df)
for i in range(int(negative_df.shape[0]/positive_df.shape[0])):
    df = pd.concat([df, positive_df])
x, y = split_xy(df)

In [26]:
best_f1 = 0.0
for trees in num_trees:
    for crit in criterion:
        for max_feat in max_features:
            model = RandomForestClassifier(n_estimators=trees, criterion=crit, max_features=max_feat)
            scores = cross_val_score(model, x, y, cv=num_folds, scoring='f1')
            f1 = np.mean(scores)
            if f1 > best_f1:
                best_f1 = f1
                best_num_trees = trees
                best_criterion = crit
                best_max_feat = max_feat
print("F1 score del mejor modelo: " + str(best_f1))
print("Numero de arboles empleados: " + str(best_num_trees))
print("Criterio de corte empleado: " + str(best_criterion))
print("Numero maximo de variables por arbol: " + str(best_max_feat))

F1 score del mejor modelo: 0.9986485869515592
Numero de arboles empleados: 100
Criterio de corte empleado: entropy
Numero maximo de variables por arbol: log2
